#Viewing finetuning curves

###Mount drive and Authenticate for GCP & Copy tfevents from GCS into drive

In [ ]:
from google.colab import auth,drive
print("Mount drive:")
drive.mount('/content/drive', force_remount=True)
DRIVE_PATH="/content/drive/My Drive"
import os
print("Authenticate for GCS:")
auth.authenticate_user()

BUCKET_NAME = "theodore_jiang" #@param{type:"string"}
TFEVENTs_DESTINATION_PATH = "/content/drive/My Drive" #@param{type:"string"}

runs = ["mrpc_loss_spam_model_comparison_final",
        "re_loss_spam_model_comparison_final",
        "ner_loss_spam_model_comparison_final"]

for run in runs: ##This will copy all of the tfevent files from GCS into drive, and will also delete all of the files from GCS
  cmd = "gsutil -m cp -r \"gs://"+BUCKET_NAME+"/"+run+"\" \""+TFEVENTs_DESTINATION_PATH+"\""
  !{cmd}
  cmd = "gsutil -m rm -r "+"\"gs://"+BUCKET_NAME+"/"+run+"\""
  !{cmd}

  

###Just mount drive (use this if you wish to only use existing tfevent files from drive)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
DRIVE_PATH = "/content/drive/My Drive"
TFEVENTs_DESTINATION_PATH = "/content/drive/My Drive" #@param{type:"string"}

###Obtain events from tfevents for finetuning graphs


In [ ]:
import os
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

from collections import defaultdict
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

runs = ["mrpc_loss_spam_model_comparison_final",
        "re_loss_spam_model_comparison_final",
        "ner_loss_spam_model_comparison_final"]

#@markdown ######only the (skip)-th tfevent file will be considered (useful when amount of tfevent files is more than what is necessary for viewing curves) (if no tfevents should be skipped, set to 1)
skip = 500 #@param {type:"integer"}

def tabulate_events(dpath,out_dict = {}):
    for dname in tqdm(os.listdir(dpath),"converting to dict"):
        if "tfevents" not in dname:
          continue
        ea = EventAccumulator(os.path.join(dpath, dname)).Reload()
        tags = ea.Tags()['scalars']

        for tag in tags:
            tag_values=[]
            wall_time=[]
            steps=[]

            for event in ea.Scalars(tag):
                 if event.step%skip==0 or "eval" in tag:
                    try:
                        out_dict[tag].append((event.step,event.value))
                    except:
                        out_dict[tag] = [(event.step,event.value)]
    return out_dict

graph_data = {}

for run in runs:
    for subrun in os.listdir("/content/drive/My Drive/"+run):
      if "predictions" in subrun:
        continue
      data = tabulate_events("/content/drive/My Drive/"+run+"/"+subrun)
      print(subrun,subrun.count("_"))
      for metric,datapoints in tqdm(data.items(),run+"_"+subrun):
          try:
              graph_data[metric][run+"-"+subrun].append(datapoints)
          except:
              try:
                  graph_data[metric][subrun] = [datapoints]
              except:
                  graph_data[metric] = {}
                  graph_data[metric][subrun] = [datapoints]



###Plotting smoothed average curves using matplotlib

In [ ]:
import matplotlib.pyplot as plt
import math
import re

#@markdown range of the local average for viewing training graphs (to disable local averaging, set it to 0)
avg_range = 100 #@param {type:"integer"}
#@markdown destination path for saving graphs (for no saving set to None)
outfolder = DRIVE_PATH+"/training graphs" #@param

if not os.path.exists(outfolder):
  os.makedirs(outfolder)

for metric,runs in graph_data.items():
    if "rando" in metric:
      continue
    plt.figure(figsize=(10,5))
    plt.title(metric+" graph")
    plt.xlabel("steps")
    plt.ylabel(metric)
    for run,run_data in runs.items():
        run_data = run_data[0]
        steps = []
        values = []
        nan = 0
        for datapt in run_data:
            if not math.isnan(datapt[1]):
                values.append(datapt[1])
                steps.append(datapt[0])
            else:
                nan+=1
        print("nan values:",nan)
        values = [x for _, x in sorted(zip(steps, values), key=lambda pair: pair[0])]

        if "eval" in metric or avg_range == 0:
          avged_values = values
        else:
          avged_values = [sum(values[max(n-avg_range,0):min(n+avg_range,len(values))])/len(values[max(n-avg_range,0):min(n+avg_range,len(values))]) for n,value in enumerate(values)]
        steps = sorted(steps)
        plt.plot(steps,avged_values,label=run)
                
    plt.legend()
    if outfolder:
      plt.savefig(outfolder+"/"+metric.replace("/","_")+".png")
    plt.show()


###Tensorboard viewing (If you wish to use tensorboard instead)

In [ ]:
LOGS_DIR = "/content/drive/My Drive" #@param (type:"string")
LOGS_DIR = "\""+LOGS_DIR+"\""
%load_ext tensorboard
%tensorboard --logdir $LOGS_DIR

#Predictions Processing

###Download prediction files from GCS (if used EVALUATE_WHILE_PREDICT)

In [ ]:
from google.colab import auth,drive
print("Mount drive:")
drive.mount('/content/drive', force_remount=True)
DRIVE_PATH="/content/drive/My Drive"
import os
print("Authenticate for GCS:")
auth.authenticate_user()

BUCKET_NAME = "theodore_jiang" #@param{type:"string"}
#@markdown where to download the tfevent files into
TFEVENTs_DESTINATION_PATH = "/content/drive/My Drive" #@param{type:"string"}

runs = ["ner_loss_spam_model_comparison_final_predictions",
        "mrpc_loss_spam_model_comparison_final_predictions"]

for run in runs[1:]: ##This will copy all of the tfevent files from GCS into drive, and will also delete all of the files from GCS
  cmd = "gsutil -m cp -r \"gs://"+BUCKET_NAME+"/"+run+"\" \""+TFEVENTs_DESTINATION_PATH+"\""
  !{cmd}
  cmd = "gsutil -m rm -r "+"\"gs://"+BUCKET_NAME+"/"+run+"\""
  !{cmd}

  

###Convert tfevents into txts (if used EVALUATE_WHILE_PREDICT)

In [ ]:
import os
from tqdm import tqdm
import re

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

runs = ["ner_loss_spam_model_comparison_final_predictions",
        "mrpc_loss_spam_model_comparison_final_predictions"]


def tabulate_events(dpath):
    out_dict = {}
    stepsa = []
    for dname in os.listdir(dpath):
      if "tfevents" not in dname:
        continue
      ea = EventAccumulator(os.path.join(dpath, dname)).Reload()
      tags = ea.Tags()['scalars']

      out = {}

      for tag in tags:
          steps=[]
          
          for event in ea.Scalars(tag):
            if not event.step in steps:
              steps.append(event.step)

          for n,event in enumerate(ea.Scalars(tag)):
            if event.step not in stepsa:
              stepsa.append(event.step)
            try:
                out_dict[tag].append((event.value,n,event.step))
            except:
                out_dict[tag] = [(event.value,n,event.step)]
    return out_dict

graph_data = {}

for run in runs:
    for subrun in os.listdir(TFEVENTs_DESTINATION_PATH+"/"+run):
      if not os.path.isdir(TFEVENTs_DESTINATION_PATH+"/"+run+"/"+subrun):
        continue
      data = tabulate_events(TFEVENTs_DESTINATION_PATH+"/"+run+"/"+subrun)
      data_polished = {}
      for k,v in data.items():
        key=re.sub("\_\d+$","",k)
        for value in v:
          try:
            data_polished[key].append(value)
          except:
            data_polished[key] = [value]
      pred_probs = [x[0] for x in sorted(sorted(list(data_polished["probability"]),key=lambda x:x[2]),key=lambda x:x[1])]
      labels = [x[0] for x in sorted(sorted(list(data_polished["label"]),key=lambda x:x[2]),key=lambda x:x[1])]
      input_ids = [x[0] for x in sorted(sorted(list(data_polished["input_id"]),key=lambda x:x[2]),key=lambda x:x[1])]

      pred_probs = pred_probs[:min(len(input_ids),len(labels),len(pred_probs))]
      labels = labels[:min(len(input_ids),len(labels),len(pred_probs))]
      input_ids = input_ids[:min(len(input_ids),len(labels),len(pred_probs))]

      with open(TFEVENTs_DESTINATION_PATH+"/"+run+"/"+subrun+"_predictions.txt","w+") as out:
        for n,prob in enumerate(tqdm(pred_probs,run+"_"+subrun)):
          out.write("probabilities:"+str(prob)+"\t"+
                    "labels:"+str(labels[n])+"\t"+
                    "input_ids:"+str(input_ids[n])+"\n")

###Just Mount drive (if not used EVALUATE_WHILE_PREDICT)

In [ ]:
from google.colab import drive,auth
import os
import shutil
!fusermount -u /content/drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)
TFEVENTs_DESTINATION_PATH = "/content/drive/My Drive"
DRIVE_PATH = "/content/drive/My Drive"

###Plot ROC Curves using txts

In [ ]:
from tqdm import tqdm
import os
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

runs = ["ner_loss_spam_model_comparison_final_predictions",  ## where the predictions were stored (should match up with 
        "mrpc_loss_spam_model_comparison_final_predictions",
        "re_loss_spam_model_comparison_final_predictions"]  ## the PREDICTIONS_FOLDER parameter in the eval script)
#@markdown destination path for saving ROC curves (for no saving set to None)
outfolder = DRIVE_PATH+"/ROC curves final" #@param
if outfolder and not os.path.exists(outfolder):
  os.makedirs(outfolder)

AUCs = {}
for run in runs:
  plt.figure(figsize=(20,10))
  for subrun in os.listdir(TFEVENTs_DESTINATION_PATH+"/"+run):
    
    labels = []
    pred_probs = []
    tp=0
    tn=0
    fp=0
    fn=0
    if os.path.isdir(TFEVENTs_DESTINATION_PATH+"/"+run+"/"+subrun):
      continue
    print("Stats for:",subrun,"\n")

    for n,line in enumerate(open(TFEVENTs_DESTINATION_PATH+"/"+run+"/"+subrun).read().split("\n")):
      line_dict = {}
      try:
        for item in line.split("\t"):
            line_dict[item.split(":")[0]] = item.split(":")[1]
        label = float(line_dict["labels"])
        pred = float(line_dict["probabilities"])
        pred_probs.append(pred)
        labels.append(label)
        if label == 1 and pred>0.5:
          tp+=1
        elif label == 0 and pred<0.5:
          tn+=1
        elif label == 0 and pred>0.5:
          fp+=1
        elif label == 1 and pred<0.5:
          fn+=1
      except Exception as e:
        print(e,line,n)
        continue

    try:
      acc = (tp+tn)/(tp+tn+fp+fn)
      recall = tp/(tp+fn)
      precision = tp/(tp+fp)
      f1 = 2*precision*recall/(precision+recall)
      pred_auc = roc_auc_score(labels, pred_probs)
    except:
      pass

    print("tp:",tp,
          "tn:",tn,
          "fp:",fp,
          "fn:",fn)
    print("acc:",acc)
    print("recall_total:",recall)
    print("precision_total:",precision)
    print("f1_total:",f1,"\n")

    # summarize scores
    # calculate roc curves
    pred_fpr, pred_tpr, _ = roc_curve(labels, pred_probs)
    # plot the roc curve for the model
    plt.plot(pred_fpr, pred_tpr, linestyle="-", label=subrun+': Area under curve: '+str(round(pred_auc,3)))
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    # show the plot
    AUCs[subrun] = round(pred_auc,3)
  plt.title("ROC for: "+run)
  if outfolder:
      plt.savefig(outfolder+"/"+run.replace(".txt",".png"))
  plt.show()

AUCs = {k:v for k,v in sorted([(k,v) for k,v in AUCs.items()],key=lambda x:x[1])}
print("Printing all AUCs...")
for k,v in AUCs.items():
  print("model:",k,"\tAUC:",v)